In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
df = pd.read_csv('drive/My Drive/DS Bootcamp 14-15.05/Zaawansowane problemy klasyfikacji/dataset/whisky.csv')
df.head()

,Distillery,Body,Sweetness,Smoky,Medicinal,Tobacco,Honey,Spicy,Winey,Nutty,Malty,Fruity,Floral,Postcode,LatitudeUTM,LongitudeUTM,Lat,Lon,Region
0,Aberfeldy,2,2,2,0,0,2,1,2,2,2,2,2,PH15 2EB,286580,749680,56.62508,-3.85052,Highland
1,Aberlour,3,3,1,0,0,4,3,2,2,3,3,2,AB38 9PJ,326340,842570,57.46728,-3.22997,Speyside
2,AnCnoc,1,3,2,0,0,2,0,0,2,2,3,2,AB54 5LI,352960,839320,57.56370,-2.75890,Highland
3,Ardbeg,4,1,4,4,0,0,2,0,1,2,1,0,PA42 7EB,141560,646220,55.64018,-6.10911,Islay & Islands
4,Ardmore,2,2,2,0,0,1,1,1,2,3,1,1,AB54 4NH,355350,829140,57.35044,-2.74388,Highland


In [4]:
from sklearn.preprocessing import  StandardScaler, MinMaxScaler

def preprocess_data(X, preprocessor=None):
    if preprocessor is not None:
        preprocessor.fit(X)
        X_preprocessed = X.copy()
        X_preprocessed[:] = preprocessor.transform(X.values)
        X = X_preprocessed
    return X

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# average:
#   None - wyniki dla każdej klasy osobno
#   'binary' - dla wybranej w `pos_label` etykiety
#   'micro' - dla całkowitej liczby TP, FN, FP
#   'macro' - nieważona średnia dla każdej klasy
#   'weighted' - ważona średnia dla każdej klasy

def calculate_metrics(target, prediction, average='macro'):
    accuracy = accuracy_score(target, prediction)
    precision = precision_score(target, prediction, average=average)
    recall = recall_score(target, prediction, average=average)
    f1 = f1_score(target, prediction, average=average)
    mislabeled = (target != prediction).sum()
    total = len(target)
    return accuracy, precision, recall, f1, mislabeled, total

def print_results(metrics, classifier_id='classifier'):
    print(f'Results for {classifier_id}')
    print('----')
    print(f'  Accuracy:  {metrics[0]}')
    print(f'  Precision: {metrics[1]}')
    print(f'  Recall:    {metrics[2]}')
    print(f'  F1 score:  {metrics[3]}')
    print(f'  Mislabeled {metrics[4]} out of {metrics[5]}')
    print('\n')
    


In [6]:

import xgboost
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

le = LabelEncoder() # zamienia tekst na liczby
df = df.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')


In [13]:
X = df.iloc[:, :-1]
y = df['Region']

In [14]:
df['Region'].value_counts()

3    41
0    31
1    11
2     3
Name: Region, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(
 X, y, stratify=y, test_size=0.33, random_state=42)

In [18]:

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline([

                   ('classifier', RandomForestClassifier())])


param = {'classifier__n_estimators': [1, 10, 20,50],
             }

grid_11 = GridSearchCV(pipe, param,cv=4, return_train_score=False,verbose=True, n_jobs=-1)
grid_11.fit(X_train, y_train)
print(grid_11.best_params_)
print("Best parameter (CV score=%0.3f):" % grid_11.best_score_)

Fitting 4 folds for each of 4 candidates, totalling 16 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  UserWarning,


{'classifier__n_estimators': 20}
Best parameter (CV score=0.788):


In [21]:
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [25]:
from sklearn.metrics import f1_score
f1_score(y_pred, y_test, average='macro')
calculate_metrics(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.8275862068965517,
 0.5821428571428571,
 0.6196428571428572,
 0.5988095238095239,
 5,
 29)

In [26]:
df['Region'].value_counts()

3    41
0    31
1    11
2     3
Name: Region, dtype: int64

In [51]:
from sklearn.utils import resample
df_majority0 = df[df['Region'] == 0]
df_minority = df[df['Region'] == 3]
df_majority2= df[df['Region'] == 2]
df_majority1  = df[df['Region'] == 1]



In [59]:

# Upsample minority class
df_majority0_upsampled = resample(df_majority0, 
                                 replace=False,    # sample with replacement
                                 n_samples=31,    # to match majority class
                                 random_state=42)
df_majority1_upsampled = resample(df_majority1,
                                 replace=True,    # sample with replacement
                                 n_samples=18,    # to match majority class
                                 random_state=42) # reproducible results

df_majority2_upsampled = resample(df_majority2, 
                                 replace=True,    # sample with replacement
                                 n_samples=5,    # to match majority class
                                 random_state=42) # reproducible results
df_minority_downsampled = resample(df_minority, 
                                 replace=False,    # sample with replacement
                                 n_samples=41,    # to match majority class
                                 random_state=42) # reproducible results
 
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority0_upsampled, df_majority1_upsampled, df_majority2_upsampled, df_minority_downsampled])
 
# Display new class counts
df_upsampled['Region'].value_counts()

3    41
0    31
1    18
2     5
Name: Region, dtype: int64

In [53]:
X = df_upsampled.iloc[:, :-1]
y = df_upsampled['Region']


In [54]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(
 X, y, test_size=0.33, random_state=42)

In [55]:

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline([

                   ('classifier', RandomForestClassifier())])


param = {'classifier__n_estimators': [1, 10, 20,50],
             }

grid_11 = GridSearchCV(pipe, param,cv=5, return_train_score=False,verbose=True, n_jobs=-1)
grid_11.fit(X_train, y_train)
grid_11.best_params_
print("Best parameter (CV score=%0.3f):" % grid_11.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


Best parameter (CV score=0.888):


In [56]:
y_pred = grid_11.predict(X_test)

In [57]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_test, y_pred,average='macro')

0.9564393939393939

In [58]:
from sklearn import  metrics


models = []

models.append(('RandomForest', grid_11.best_estimator_))


precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test, model.predict(X_test),average='macro') ))
    print("recall_score: {}".format( metrics.recall_score(y_test, model.predict(X_test),average='macro') ))
    print("f1_score: {}".format( metrics.f1_score(y_train, model.predict(X_train),average='macro') ))
    print("f1_score test: {}".format( metrics.f1_score(y_test, model.predict(X_test),average='macro') ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_train, model.predict(X_train)) ))
    print("accuracy_score_test: {}".format( metrics.accuracy_score(y_test, model.predict(X_test)) ))

RandomForest
precision_score: 0.9564393939393939
recall_score: 0.9564393939393939
f1_score: 1.0
f1_score test: 0.9564393939393939
accuracy_score: 1.0
accuracy_score_test: 0.9375


Z.8. Na zbiorze glasses wykonać zrównoważnie klas  a nastepnie użyć metody Random Forest albo Xgboost zapisze wynik f1_score na zbiorze testowym.*kursywa*

In [70]:
!git clone https://github.com/matzim95/ML-datasets
import pandas as pd

def load_dataset(filename, class_column, index_col=None):
    dataset = pd.read_csv(f'ML-datasets/{filename}.csv', index_col=index_col)
    dataset['class'] = dataset[class_column].astype('category').cat.codes
    classes = dataset.pop(class_column).unique()
    return dataset, classes

def prepare_dataset(dataset_name):
    params = {'iris': {'class_column': 'species', 'index_col': None},
              'wine': {'class_column': 'Class', 'index_col': None},
              'glass': {'class_column': 'Type', 'index_col': 'ID'},
              'breast_cancer' : {'class_column' : 'Class', 'index_col': 'Sample code number'}}
    dataset, classes = load_dataset(dataset_name,
                                    **params[dataset_name])
    y = dataset.pop('class')
    X = dataset
    return X, y, classes

fatal: destination path 'ML-datasets' already exists and is not an empty directory.


In [71]:
X, y, classes = prepare_dataset('glass')

In [72]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Normalizer, StandardScaler

normalizer = Normalizer()
standarizer = StandardScaler()
preprocessor = standarizer


In [73]:

def preprocess_data(X, preprocessor=None):
    if preprocessor is not None:
        preprocessor.fit(X)
        X_preprocessed = X.copy()
        X_preprocessed[:] = preprocessor.transform(X.values)
        X = X_preprocessed
    return X

In [74]:
X = preprocess_data(X, standarizer)
X.describe()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron
count,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02,2.140000e+02
mean,-2.877449e-14,2.179980e-15,-2.801497e-16,-3.434428e-16,9.966067e-16,7.470660e-17,-3.137418e-16,-1.763906e-16,-6.121791e-17
std,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00,1.002345e+00
min,-2.381516e+00,-3.286943e+00,-1.865511e+00,-2.318616e+00,-3.676472e+00,-7.639186e-01,-2.484084e+00,-3.528768e-01,-5.864509e-01
25%,-6.082728e-01,-6.141580e-01,-3.957744e-01,-5.117560e-01,-4.800288e-01,-5.756501e-01,-5.049657e-01,-3.528768e-01,-5.864509e-01
50%,-2.262293e-01,-1.323817e-01,5.527787e-01,-1.704602e-01,1.799655e-01,8.905322e-02,-2.514132e-01,-3.528768e-01,-5.864509e-01
75%,2.614331e-01,5.120326e-01,6.361680e-01,3.715977e-01,5.649621e-01,1.735820e-01,1.518057e-01,-3.528768e-01,4.422417e-01
max,5.137232e+00,4.875637e+00,1.254639e+00,4.125851e+00,3.570524e+00,8.780145e+00,5.094318e+00,5.997210e+00,4.659881e+00


In [75]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(
 X, y, stratify=y, test_size=0.33, random_state=42)

In [76]:

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline([

                   ('classifier', RandomForestClassifier())])


param = {'classifier__n_estimators': [1, 10, 20,50],
             }

grid_11 = GridSearchCV(pipe, param,cv=5, return_train_score=False,verbose=True, n_jobs=-1)
grid_11.fit(X_train, y_train)
grid_11.best_params_
print("Best parameter (CV score=%0.3f):" % grid_11.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameter (CV score=0.747):


In [77]:
y_pred = grid_11.predict(X_test)

In [78]:
from sklearn import  metrics


models = []

models.append(('RandomForest', grid_11.best_estimator_))


precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test, model.predict(X_test),average='macro') ))
    print("recall_score: {}".format( metrics.recall_score(y_test, model.predict(X_test),average='macro') ))
    print("f1_score: {}".format( metrics.f1_score(y_train, model.predict(X_train),average='macro') ))
    print("f1_score test: {}".format( metrics.f1_score(y_test, model.predict(X_test),average='macro') ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_train, model.predict(X_train)) ))
    print("accuracy_score_test: {}".format( metrics.accuracy_score(y_test, model.predict(X_test)) ))

RandomForest
precision_score: 0.7488606573906029
recall_score: 0.7493719806763285
f1_score: 0.9903731082567716
f1_score test: 0.6763292263292263
accuracy_score: 0.986013986013986
accuracy_score_test: 0.7183098591549296


In [79]:
y.value_counts()

1    76
0    70
3    29
5    17
2    13
4     9
Name: class, dtype: int64

In [99]:
y_new = pd.concat([X,y], axis=1) #aby osiągnąc surowy dataframe glass

In [100]:
#wyodrebnienie konkretnych danych dla konkretnej wartości etykiety
from sklearn.utils import resample
df_majority0 = y_new[y_new['class'] == 0]
df_minority = y_new[y_new['class'] == 3]
df_majority2= y_new[y_new['class'] == 2]
df_majority1  = y_new[y_new['class'] == 1]
df_majority3 = y_new[y_new['class'] == 4]
df_majority4 = y_new[y_new['class'] == 5]


In [101]:
df_majority0

,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,class
ID,,,,,,,,,,
1,0.872868,0.284953,1.254639,-0.692442,-1.127082,-0.671705,-0.145766,-0.352877,-0.586451,0
2,-0.249333,0.591817,0.636168,-0.170460,0.102319,-0.026213,-0.793734,-0.352877,-0.586451,0
3,-0.721318,0.149933,0.601422,0.190912,0.438787,-0.164533,-0.828949,-0.352877,-0.586451,0
4,-0.232831,-0.242853,0.698710,-0.310994,-0.052974,0.112107,-0.519052,-0.352877,-0.586451,0
5,-0.312045,-0.169205,0.650066,-0.411375,0.555256,0.081369,-0.624699,-0.352877,-0.586451,0
...,...,...,...,...,...,...,...,...,...,...
66,0.866266,0.346326,0.629219,-0.652289,-0.894143,-0.625599,0.312037,-0.352877,-0.586451,0
67,1.041198,-0.439246,0.670914,-1.154195,-0.557675,-0.471910,0.628977,-0.352877,1.162326,0
68,1.041198,-0.439246,0.670914,-1.154195,-0.428265,-0.471910,0.628977,-0.352877,1.162326,0


In [102]:
#dolosowywanie danych
# Upsample minority class
df_minority0_downsampled = resample(df_majority0, 
                                 replace=False,    # sample with replacement
                                 n_samples=70,    # to match majority class
                                 random_state=42)
df_minority1_downsampled= resample(df_majority1,
                                 replace=False,    # sample with replacement
                                 n_samples=76,    # to match majority class
                                 random_state=42) # reproducible results

df_majority2_upsampled = resample(df_majority2, 
                                 replace=True,    # sample with replacement
                                 n_samples=25,    # to match majority class
                                 random_state=42) # reproducible results
df_majority3_upsampled = resample(df_majority3, 
                                 replace=True,    # sample with replacement
                                 n_samples=15,    # to match majority class
                                 random_state=42) # reproducible results
df_majority4_upsampled = resample(df_majority4, 
                                 replace=True,    # sample with replacement
                                 n_samples=30,    # to match majority class
                                 random_state=42) # reproducible results                                                                 
df_majority5_upsampled= resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples=35,    # to match majority class
                                 random_state=42) # reproducible results
 
 
# Combine majority class with upsampled minority class
df_glass_upsampled = pd.concat([df_minority0_downsampled, df_minority1_downsampled, df_majority2_upsampled, df_majority3_upsampled, df_majority4_upsampled, df_majority5_upsampled])
 
# Display new class counts
df_glass_upsampled.head()

,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,class
ID,,,,,,,,,,
23,-0.331848,-0.770659,0.650066,-0.310994,0.179965,0.142844,-0.180982,-0.352877,-0.586451,0
1,0.872868,0.284953,1.254639,-0.692442,-1.127082,-0.671705,-0.145766,-0.352877,-0.586451,0
50,0.202848,0.211306,0.462440,-0.431451,-0.738850,0.142844,-0.033076,-0.352877,-0.586451,0
5,-0.312045,-0.169205,0.650066,-0.411375,0.555256,0.081369,-0.624699,-0.352877,-0.586451,0
55,-0.193223,-0.242853,0.087189,-0.310994,0.425846,0.019893,0.044398,-0.352877,0.339372,0


In [105]:
#podział na cechy i etykiety
y = df_glass_upsampled['class']
X = df_glass_upsampled.iloc[: , :9]

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(
 X, y, stratify=y, test_size=0.33, random_state=42)

In [109]:

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline([

                   ('classifier', RandomForestClassifier())])


param = {'classifier__n_estimators': [1, 10, 20,50],
             }

grid_11 = GridSearchCV(pipe, param,cv=5, return_train_score=False,verbose=True, n_jobs=-1)
grid_11.fit(X_train, y_train)
grid_11.best_params_
print("Best parameter (CV score=%0.3f):" % grid_11.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameter (CV score=0.816):


In [111]:
y_pred = grid_11.predict(X_test)

In [112]:
from sklearn import  metrics


models = []

models.append(('RandomForest', grid_11.best_estimator_))


precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test, model.predict(X_test),average='macro') ))
    print("recall_score: {}".format( metrics.recall_score(y_test, model.predict(X_test),average='macro') ))
    print("f1_score: {}".format( metrics.f1_score(y_train, model.predict(X_train),average='macro') ))
    print("f1_score test: {}".format( metrics.f1_score(y_test, model.predict(X_test),average='macro') ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_train, model.predict(X_train)) ))
    print("accuracy_score_test: {}".format( metrics.accuracy_score(y_test, model.predict(X_test)) ))

RandomForest
precision_score: 0.8514619883040937
recall_score: 0.8826086956521739
f1_score: 1.0
f1_score test: 0.8630110630110631
accuracy_score: 1.0
accuracy_score_test: 0.8313253012048193
